<div style="border: 2px solid #00008B; padding: 15px; border-radius: 10px; background-color: #00008B; color: #FFFFFF; font-family: Arial;">
  <h1 style="margin-top: 0;">Prática: Reconhecimento de Entidades Nomeadas (Named Entity Recognition - NER)</h1>
</div>

# Biblioteca Spacy

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
Uma das bibliotecas mais poderosas e eficientes para tarefas de NLP é a spaCy. SpaCy é uma biblioteca Python de código aberto projetada para lidar com tarefas complexas de NLP de maneira eficiente. 

https://medium.com/@flaviagaia/desbravando-os-comandos-da-biblioteca-spacy-para-processamento-de-linguagem-natural-19061d5f7586

</div>

<h1><span style="color:red">Neste exemplo vamos usar o GOOGLE COLAB.</span></h1>

Faça Upload desse Notebook no https://colab.research.google.com/

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download pt_core_news_sm
!python -m spacy download pt_core_news_lg

In [ ]:
import spacy
from spacy import displacy
from google.colab import drive, files

# Montando o Drive

In [ ]:
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/LlamaModels'
model_path = '/content/drive/MyDrive/LlamaModels'

# Carga de dados

In [ ]:
with open("/content/drive/MyDrive/LlamaModels/decisao.txt", "r") as file1:
    dataset = file1.read()
print(dataset)

In [ ]:
#with open("../data/decisao.txt", "r", encoding="utf-8") as f:
#    dataset = f.read()
#print(dataset)

# Usando o Spacy

In [ ]:
# Carregar o modelo de linguagem em português
nlp = spacy.load("pt_core_news_lg")

In [ ]:
# Processar o texto
doc = nlp(dataset)

In [ ]:
# Reconhecer entidades
for entidade in doc.ents:
    print(entidade.text, entidade.label_)

In [ ]:
# Visualizar as entidades nomeadas
displacy.render(doc, style="ent", jupyter=True)

In [ ]:

# Função para extrair entidades baseadas em regras simples
def extrair_entidades(doc):
    entidades = []
    for token in doc:
        # Regra para números de processos
        if "processo" in token.text.lower() and token.i + 1 < len(doc) and doc[token.i + 1].is_digit:
            entidades.append({"tipo": "Número do Processo", "entidade": doc[token.i + 1].text})
        # Regra para classes judiciais
        if "classe judicial" in token.text.lower():
            entidades.append({"tipo": "Classe Judicial", "entidade": doc[token.i + 2].text})
        # Regra para requerente e requerido
        if token.text.lower() == "requerente":
            entidade = " ".join([w.text for w in doc[token.i + 2: token.i + 5]])
            entidades.append({"tipo": "Requerente", "entidade": entidade})
        if token.text.lower() == "requerido":
            entidade = " ".join([w.text for w in doc[token.i + 2: token.i + 5]])
            entidades.append({"tipo": "Requerido", "entidade": entidade})
        # Regra para datas
        if token.like_num and (token.nbor(1).text == "dias" or token.nbor(1).text == "anos"):
            entidades.append({"tipo": "Tempo", "entidade": token.text + " " + token.nbor(1).text})
        # Regra para localidades
        if token.text in ["Brasília", "Goiânia"]:
            entidades.append({"tipo": "Localidade", "entidade": token.text})

    return entidades


In [ ]:
entidades = extrair_entidades(doc)
for entidade in entidades:
    print(f"Tipo: {entidade['tipo']}, Entidade: {entidade['entidade']}")